#QUESTION 2: Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World



Now that you are having a basic knowledge of how the agent must work, recall our discussions regarding the agent types and environment. We will now include the PEAS information and redesign the agent.
In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [ ]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [ ]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'suck'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a performance measure which is here the number of cleaned squares (since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room as required in the PEAS description above). The energy budget of the agent is specified as `max_steps`.

In [ ]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0

    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action)

        if (action == "suck"):
            num_cleaned = num_cleaned + 1

    return num_cleaned



Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [ ]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: north
step 1 - action: east
step 2 - action: south
step 3 - action: west
step 4 - action: suck
step 5 - action: east
step 6 - action: suck
step 7 - action: north
step 8 - action: south
step 9 - action: east
step 10 - action: south
step 11 - action: west
step 12 - action: west
step 13 - action: west
step 14 - action: suck
step 15 - action: north
step 16 - action: suck
step 17 - action: east
step 18 - action: north
step 19 - action: east


4

# Tasks


1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision.
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement intelligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formatted professionally.
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. **Use the same environment for all your agent implementations in the tasks below.**

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [ ]:
# Your code and description goes here

# importing required modules
import numpy as np

def simple_environment(n, verbose=True):
    c = 0
    e = 0
    env = np.random.randint(2, size=(n, n)) # Generate a random 2D Environmnet with values 0 or 1
    # env = np.random.choice([0, 1], size=(n, n), p=[0.5, 0.5])
    # env = np.array([[int(x) for x in input("Enter row: ").split()]for i in range(n)])
    print("\nEnvironment:",env,sep="\n")
    print()

    def simple_reflex_agent(env, c, e):
        n = len(env)
        visited = set() # initialize set to keep track of visit
        # Generate a random position for the agent in a 2D space
        i = np.random.randint(n)
        j = np.random.randint(n)

        def agent():
          actions = ["north", "east", "south", "west"] # Actions for robot
          np.random.shuffle(actions) # Shuffling actions
          return actions

        def dfs(row, col, action, c, e):
            nonlocal verbose

            if 0 <= row < n and 0 <= col < n and verbose == True and (row, col) not in visited: # condition check
                visited.add((row, col)) # adding as visited
                print(f"Step {e} - Position: ({row},{col}), Action: {action}")    # display
                print("Location is Dirty, Action: suck") if env[row,col] == 1 else print("Location is Clean.\n")   # Check dirty or clean and display

                if env[row,col] == 1:
                    env[row,col] = 0
                    c += 1 # increment
                    print("Location has been Cleaned.\n") # display
                e += 1 # increment

                actions = agent() # get an action
                for next_action in actions:
                    # Condition check for particular actions in random order and its dfs call
                    if next_action == "north" and row > 0:
                        c, e = dfs(row - 1, col, "north", c, e)
                    elif next_action == "south" and row < n - 1:
                        c, e = dfs(row + 1, col, "south", c, e)
                    elif next_action == "west" and col > 0:
                        c, e = dfs(row, col - 1, "west", c, e)
                    elif next_action == "east" and col < n - 1:
                        c, e = dfs(row, col + 1, "east", c, e)
            return c, e  # return statemnet

        return dfs(i, j, "start", c, e)  # dfs fn call output as return

    c, e = simple_reflex_agent(env, c, e) # fn call
    print("Environment after cleaning:",env,sep="\n")
    print()

    return c, e # return statemnet


n = int(input("Enter the size of the room (nxn): ")) # take user input

c, e = simple_environment(n) # fn call

# display results
print("Total cleaned locations:",c)
print("Total energy used:",e)


Enter the size of the room (nxn): 3

Environment:
[[1 1 0]
 [1 0 0]
 [1 1 1]]

Step 0 - Position: (1,2), Action: start
Location is Clean.

Step 1 - Position: (0,2), Action: north
Location is Clean.

Step 2 - Position: (0,1), Action: west
Location is Dirty, Action: suck
Location has been Cleaned.

Step 3 - Position: (0,0), Action: west
Location is Dirty, Action: suck
Location has been Cleaned.

Step 4 - Position: (1,0), Action: south
Location is Dirty, Action: suck
Location has been Cleaned.

Step 5 - Position: (1,1), Action: east
Location is Clean.

Step 6 - Position: (2,1), Action: south
Location is Dirty, Action: suck
Location has been Cleaned.

Step 7 - Position: (2,2), Action: east
Location is Dirty, Action: suck
Location has been Cleaned.

Step 8 - Position: (2,0), Action: west
Location is Dirty, Action: suck
Location has been Cleaned.

Environment after cleaning:
[[0 0 0]
 [0 0 0]
 [0 0 0]]

Total cleaned locations: 6
Total energy used: 9




The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function.

In [ ]:
# Your code and description goes here

# importing required modules
import random
import numpy as np

def simple_environment(n,prob):
    #env = np.array([[random.randint(0, 1) for i in range(n)] for j in range(n)]) # Generate a random 2D array with values 0 or 1
    env = np.random.choice([0, 1], size=(n, n), p=[prob, 1-prob])
    # env = np.array([[int(x) for x in input("Enter row: ").split()]for i in range(n)])
    # display
    print("\nEnvironment:",env,sep="\n")
    print()

    def simple_randomized_agent(env,verbose=True):
        n = len(env) # get len
        visited = set() # initialize set to keep track of visit

        def visit(i, j):
            visited.add((i, j)) # adding as visited
            if env[i,j] == 1: # check clean or not and display action
                print("Action: suck")
                env[i,j] = 0
                return 1
            else: return 0

        def agent():
          return random.choice(["north", "east", "south", "west"])  # returns random action

        i, j = np.random.randint(n), np.random.randint(n) # Generate a random position for the agent in a 2D space
        c = 0
        s = 0
        while len(visited) < n * n and verbose==True:  # looping till all rooms visited
            print("Step",c,"Position:", f"({i},{j})", end=" ") # display
            c += 1
            d = visit(i, j) # fn call
            s += d
            if d == 1:
                print("Step",c,"Position:", f"({i},{j})", end=" ")  # display
                c += 1
            while 1:
              move = agent() # get random move for agent
              # Condition check for particular moves
              if move == "north" and i > 0:
                  i -= 1
                  break
              elif move == "east" and j < n - 1:
                  j += 1
                  break
              elif move == "south" and i < n - 1:
                  i += 1
                  break
              elif move == "west" and j > 0:
                  j -= 1
                  break
            print("Action:", move) # display

        return c,s

    c,s = simple_randomized_agent(env) # # fn call

    # display
    print("\nEnvironment after cleaning:",env,sep="\n")
    print()

    return c,s

size = int(input("Enter matrix dimension: "))
p = float(input("Enter probability: "))
enery_used, cleaned_loc = simple_environment(size,p)


print("Total locations Cleaned =",cleaned_loc)
print("Total energy Used =",enery_used)



Enter matrix dimension: 3
Enter probability: 0.2

Environment:
[[1 1 1]
 [1 0 1]
 [1 1 1]]

Step 0 Position: (1,1) Action: east
Step 1 Position: (1,2) Action: suck
Step 2 Position: (1,2) Action: south
Step 3 Position: (2,2) Action: suck
Step 4 Position: (2,2) Action: north
Step 5 Position: (1,2) Action: west
Step 6 Position: (1,1) Action: south
Step 7 Position: (2,1) Action: suck
Step 8 Position: (2,1) Action: north
Step 9 Position: (1,1) Action: west
Step 10 Position: (1,0) Action: suck
Step 11 Position: (1,0) Action: north
Step 12 Position: (0,0) Action: suck
Step 13 Position: (0,0) Action: south
Step 14 Position: (1,0) Action: east
Step 15 Position: (1,1) Action: south
Step 16 Position: (2,1) Action: west
Step 17 Position: (2,0) Action: suck
Step 18 Position: (2,0) Action: north
Step 19 Position: (1,0) Action: east
Step 20 Position: (1,1) Action: west
Step 21 Position: (1,0) Action: east
Step 22 Position: (1,1) Action: west
Step 23 Position: (1,0) Action: north
Step 24 Position: (0,